In [ ]:
# target_encoding

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

# 各columns(catの英数字)毎にtargetの数値を平均して、特徴量に加える
for col in object_cols:
    temp_df = []
    temp_test_feat = None
    for fold in range(5):
        xtrain =  df[df.kfold != fold].reset_index(drop=True)
        xvalid = df[df.kfold == fold].reset_index(drop=True)

        # col(A, Bなど)毎にtarget(目的変数)の平均を出す
        feat = xtrain.groupby(col)["target"].agg('mean')
        feat = feat.to_dict()
        
        #xvalidにtargetの平均columを加える
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_df.append(xvalid)
        
        # 値がtemp_test_featに入っていたら、targetの平均値を加える
        if temp_test_feat is None:
            temp_test_feat = df_test[col].map(feat)
        else:
            temp_test_feat += df_test[col].map(feat)
            
    #5回繰り返したので、5で割る
    temp_test_feat /= 5
    df_test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    df = pd.concat(temp_df)